## History

In [15]:
import pandas as pd
datapath = "tuning/"
test_recommendations = pd.read_csv(datapath + "test_recommendations.csv")
train_recommendations = pd.read_csv(datapath + "train_recommendations.csv")
app_plots = pd.read_csv(datapath + "apps_fixed2.csv")
app_plots.columns, train_recommendations.columns, test_recommendations.columns

(Index(['authors', 'average_rating', 'app_id', 'books_count', 'description',
        'genres', 'year', 'original_title', 'pages', 'ratings_count', 'title',
        'authors_2', 'new_title'],
       dtype='object'),
 Index(['user_id', 'app_id', 'rating'], dtype='object'),
 Index(['user_id', 'app_id', 'rating'], dtype='object'))

In [16]:
app_plots.drop(columns=["title"], inplace=True)
app_plots.rename(columns={"description":"desc", "average_rating": "Rating", "new_title": "title"}, inplace=True)

In [ ]:
app_plots.rename(columns={"description":"desc", "average_rating": "Rating", "new_title": "title"}, inplace=True)

In [9]:
app_plots

,authors,Rating,app_id,books_count,desc,genres,year,original_title,pages,ratings_count,authors_2,title
0,['Suzanne Collins'],4.34,1,272,WINNING MEANS FAME AND FORTUNE.LOSING MEANS CE...,"['young-adult', 'fiction', 'fantasy', 'science...",2008.0,The Hunger Games,374.0,4780653,['Suzanne Collins'],The Hunger Games
1,"['J.K. Rowling', 'Mary GrandPré']",4.44,2,491,Harry Potter's life is miserable. His parents ...,"['fantasy', 'fiction', 'young-adult', 'classics']",1997.0,Harry Potter and the Philosopher's Stone,309.0,4602479,"['J.K. Rowling', 'Mary GrandPré']",Harry Potter and the Philosopher's Stone
2,['Stephenie Meyer'],3.57,3,226,About three things I was absolutely positive.\...,"['young-adult', 'fantasy', 'romance', 'fiction...",2005.0,Twilight,501.0,3866839,['Stephenie Meyer'],"Twilight (Twilight, #1)"
3,['Harper Lee'],4.25,4,487,The unforgettable novel of a childhood in a sl...,"['classics', 'fiction', 'historical-fiction', ...",1960.0,To Kill a Mockingbird,324.0,3198671,['Harper Lee'],To Kill a Mockingbird
4,['F. Scott Fitzgerald'],3.89,5,1356,Alternate Cover Edition ISBN: 0743273567 (ISBN...,"['classics', 'fiction', 'historical-fiction', ...",1925.0,The Great Gatsby,200.0,2683664,['F. Scott Fitzgerald'],The Great Gatsby
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,['[Jeffery Deaver]'],3.98,9981,58,"A high-school girl in Harlem, Geneva Settle, i...","['mystery', 'thriller', 'fiction', 'crime', 's...",2005.0,The Twelfth Card,816.0,10188,['[Jeffery Deaver]'],The Twelfth Card
9996,['[Karen Marie Moning]'],4.18,9982,1,In Karen Marie Moning’s latest installment of ...,"['fantasy', 'paranormal', 'romance', 'fiction'...",2016.0,Feverborn,436.0,11570,['[Karen Marie Moning]'],Feverborn
9997,"['[Steve Perry', 'Tom Clancy', 'Steve Pieczeni...",3.69,9985,24,"In the year 2000, computers are the new superp...","['fiction', 'thriller', 'books', 'mystery', 's...",2000.0,Tom Clancy's Net Force: Breaking Point,368.0,7693,"['[Steve Perry', 'Tom Clancy', 'Steve Pieczeni...",Tom Clancy's Net Force: Breaking Point
9998,['[Iris Johansen]'],4.12,9987,29,A CIA agent's two-year-old child was stolen in...,"['mystery', 'fiction', 'thriller', 'suspense',...",2010.0,Chasing The Night,362.0,10129,['[Iris Johansen]'],Chasing The Night


In [2]:
df = pd.read_csv("tuning/test_recommendations.csv")

# 2. Calcola la popolarità normalizzata
# .value_counts(normalize=True) calcola automaticamente il rapporto (frequenza relativa)
popularity_normalized = df['app_id'].value_counts().reset_index()
popularity_normalized.columns = ['app_id', 'popularity']

# 3. Unisci la colonna di popolarità al DataFrame originale
# Viene unito in base alla colonna 'app_id'
df_with_popularity = pd.merge(df, popularity_normalized, on='app_id', how='left')

# Stampa il risultato per verifica, arrotondando la popolarità a 4 decimali per chiarezza
df_with_popularity['popularity'] = df_with_popularity['popularity'].round(4)
print(df_with_popularity)

       user_id  app_id  rating  popularity
0        17978    3465       5           3
1        22593    1198       5           1
2        43809      92       4           9
3          476    4527       5           1
4        39161    3552       3           2
...        ...     ...     ...         ...
12131    49564     392       4           2
12132    52356    1221       4           3
12133    44740    7782       3           1
12134    45422     143       4          17
12135     2749    1279       5           3

[12136 rows x 4 columns]


In [3]:
# stampa la popolarità media
average_popularity = df_with_popularity['popularity'].mean()
print(f"Popolarità media: {1/average_popularity}")

Popolarità media: 0.13094518774277084


In [23]:
df_with_popularity.to_csv("tuning/test_recommendations_with_popularity.csv", index=False)

In [17]:
merged_train_df = pd.merge(train_recommendations, app_plots, on='app_id', how='inner')

# --- 3. Seleziona e Rinomina le Colonne Necessarie ---
# Selezioniamo solo le colonne che ci servono per il file finale.
# Rinominiamo 'genres' in 'genre' e 'rating' per corrispondere alla tua richiesta.
# Nota: ho corretto il tuo "raiting" in "rating".
history_train_data = merged_train_df[[
    'user_id',
    'title',
    'desc',
    'genres', # Selezioniamo la colonna originale
    'Rating'
]]

print("Creazione delle storie degli utenti...")
user_histories_train_df = (
    history_train_data.groupby('user_id')[['title', 'desc', 'genres', 'Rating']]
                .apply(lambda x: x.to_dict('records'))
                .reset_index(name='history')
)
merged_test_df = pd.merge(test_recommendations, app_plots, on='app_id', how='inner')
history_test_data = merged_test_df[[
    'user_id',
    'title',
    'desc',
    'genres', # Selezioniamo la colonna originale
    'Rating'
]]
user_histories_test_df = (
    history_test_data.groupby('user_id')[['title', 'desc', 'genres', 'Rating']]
                .apply(lambda x: x.to_dict('records'))
                .reset_index(name='history')
)

output_filename = datapath + "histories_gemma_recommender_train.json"
print(f"Salvataggio del file JSON in '{output_filename}'...")
user_histories_train_df.to_json(output_filename, orient='records', indent=4, force_ascii=False)

output_filename = datapath + "histories_gemma_recommender_test.json"
print(f"Salvataggio del file JSON in '{output_filename}'...")
user_histories_test_df.to_json(output_filename, orient='records', indent=4, force_ascii=False)


print("✨ Fatto! File JSON creato con successo.")

Creazione delle storie degli utenti...
Salvataggio del file JSON in 'tuning/histories_gemma_recommender_train.json'...
Salvataggio del file JSON in 'tuning/histories_gemma_recommender_test.json'...
✨ Fatto! File JSON creato con successo.


In [24]:
merged_test_df = pd.merge(df_with_popularity, app_plots, on='app_id', how='inner')
merged_test_df

,user_id,app_id,Rating,Timestamp,popularity,title,tmdbId,imdbId,desc,genres
0,5950,2550,1,957180825,20,"Haunting, The (1963)",11772.0,57129.0,Dr. John Markway invites three distinct indivi...,Horror|Thriller
1,1513,2550,3,974749912,20,"Haunting, The (1963)",11772.0,57129.0,Dr. John Markway invites three distinct indivi...,Horror|Thriller
2,1941,2550,4,974850476,20,"Haunting, The (1963)",11772.0,57129.0,Dr. John Markway invites three distinct indivi...,Horror|Thriller
3,33,2550,4,978109889,20,"Haunting, The (1963)",11772.0,57129.0,Dr. John Markway invites three distinct indivi...,Horror|Thriller
4,1733,2550,4,977461820,20,"Haunting, The (1963)",11772.0,57129.0,Dr. John Markway invites three distinct indivi...,Horror|Thriller
...,...,...,...,...,...,...,...,...,...,...
108018,2167,1720,2,974612960,1,Time Tracers (1995),438108.0,128755.0,When the discovery of a five-thousand year old...,Action|Adventure|Sci-Fi
108019,5333,148,3,989050056,1,"Awfully Big Adventure, An (1995)",22279.0,112427.0,"Liverpool. 1947. Right after World War II, a s...",Drama
108020,2015,2585,1,974677124,1,"Lovers of the Arctic Circle, The (Los Amantes ...",1414.0,133363.0,Otto and Ana are kids when they meet each othe...,Drama|Romance
108021,1812,3540,4,975367635,1,Passion of Mind (1999),19214.0,160644.0,A psychological romantic thriller where fantas...,Romance|Thriller


In [25]:
merged_test_df = pd.merge(df_with_popularity, app_plots, on='app_id', how='inner')
history_test_data = merged_test_df[[
    'user_id',
    'title',
    'desc',
    'genres', # Selezioniamo la colonna originale
    'Rating',
    'popularity'  # Aggiunta della colonna di popolarità
]]
user_histories_test_df = (
    history_test_data.groupby('user_id')[['title', 'desc', 'genres', 'Rating', 'popularity']]
                .apply(lambda x: x.to_dict('records'))
                .reset_index(name='history')
)


output_filename = datapath + "histories_gemma_recommender_test_with_popularity.json"
print(f"Salvataggio del file JSON in '{output_filename}'...")
user_histories_test_df.to_json(output_filename, orient='records', indent=4, force_ascii=False)


print("✨ Fatto! File JSON creato con successo.")

Salvataggio del file JSON in 'tuning/histories_gemma_recommender_test_with_popularity.json'...
✨ Fatto! File JSON creato con successo.


## CANDIDATI

In [ ]:
import pandas as pd
datapath = "tuning/"
candidate_items = pd.read_csv(datapath + "candidate/candidate_items_10_hits.csv")
candidate_items

,user_id,app_id,authors,average_rating,books_count,plot,genres,year,new_title,pages,ratings_count,authors_2
0,167,178,['Sylvia Plath'],3.98,246,The Bell Jar chronicles the crack-up of Esther...,"['classics', 'fiction', 'psychology', 'poetry']",1963.0,The Bell Jar,294.0,401605,['Sylvia Plath']
1,8707,178,['Sylvia Plath'],3.98,246,The Bell Jar chronicles the crack-up of Esther...,"['classics', 'fiction', 'psychology', 'poetry']",1963.0,The Bell Jar,294.0,401605,['Sylvia Plath']
2,9661,178,['Sylvia Plath'],3.98,246,The Bell Jar chronicles the crack-up of Esther...,"['classics', 'fiction', 'psychology', 'poetry']",1963.0,The Bell Jar,294.0,401605,['Sylvia Plath']
3,12285,178,['Sylvia Plath'],3.98,246,The Bell Jar chronicles the crack-up of Esther...,"['classics', 'fiction', 'psychology', 'poetry']",1963.0,The Bell Jar,294.0,401605,['Sylvia Plath']
4,14950,178,['Sylvia Plath'],3.98,246,The Bell Jar chronicles the crack-up of Esther...,"['classics', 'fiction', 'psychology', 'poetry']",1963.0,The Bell Jar,294.0,401605,['Sylvia Plath']
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,51329,361,"['Charles Dickens', 'George Cruikshank', 'Phil...",3.85,1787,A gripping portrayal of London's dark criminal...,"['classics', 'fiction', 'historical-fiction']",1838.0,Oliver Twist,608.0,230181,"['Charles Dickens', 'George Cruikshank', 'Phil..."
9996,39400,280,['Lauren Oliver'],3.99,108,There is an alternate cover edition for this I...,"['young-adult', 'romance', 'science-fiction', ...",2011.0,Delirium,441.0,256975,['Lauren Oliver']
9997,28258,9718,['[Chris Cleave]'],3.48,25,When a massive suicide bomb explodes at a Lond...,"['fiction', 'contemporary', 'thriller']",2012.0,"Happier at Home: Kiss More, Jump More, Abandon...",239.0,6058,['[Chris Cleave]']
9998,27762,2243,['[A.J. Banner]'],3.33,14,From a phenomenal new voice in suspense fictio...,"['fiction', 'mystery', 'thriller', 'suspense',...",2015.0,NaN,NaN,37604,['[A.J. Banner]']


In [32]:
import pandas as pd
datapath = "tuning/"
candidate_items = pd.read_csv(datapath + "candidate_items_10_train_new.csv")
app_plots = pd.read_csv(datapath + "apps.csv")
app_plots.columns, candidate_items.columns
candidate_items.rename(columns={'item_id': 'app_id'}, inplace=True)
candidate_items = pd.merge(candidate_items, app_plots, on='app_id', how='inner')
candidate_items.to_csv(datapath + "candidate_items_10.csv", index=False)

In [16]:
candidate_items.rename(columns={'item_id': 'app_id'}, inplace=True)
candidate_items

,user_id,app_id
0,167,13
1,167,899
2,167,291
3,167,19
4,167,42
...,...,...
29995,53419,336
29996,53419,752
29997,53419,223
29998,53419,617


In [17]:
candidate_items = pd.merge(candidate_items, app_plots, on='app_id', how='inner')
candidate_items

,user_id,app_id,authors,average_rating,books_count,plot,genres,year,new_title,pages,ratings_count,authors_2
0,167,13,"['[George Orwell', 'Marcelo Pen', 'Erich Fromm...",4.14,995,"The year 1984 has come and gone, but George Or...","['classics', 'fiction', 'science-fiction', 'fa...",1949.0,Nineteen Eighty-Four,328.0,1956832,"['[George Orwell', 'Marcelo Pen', 'Erich Fromm..."
1,1055,13,"['[George Orwell', 'Marcelo Pen', 'Erich Fromm...",4.14,995,"The year 1984 has come and gone, but George Or...","['classics', 'fiction', 'science-fiction', 'fa...",1949.0,Nineteen Eighty-Four,328.0,1956832,"['[George Orwell', 'Marcelo Pen', 'Erich Fromm..."
2,1101,13,"['[George Orwell', 'Marcelo Pen', 'Erich Fromm...",4.14,995,"The year 1984 has come and gone, but George Or...","['classics', 'fiction', 'science-fiction', 'fa...",1949.0,Nineteen Eighty-Four,328.0,1956832,"['[George Orwell', 'Marcelo Pen', 'Erich Fromm..."
3,1661,13,"['[George Orwell', 'Marcelo Pen', 'Erich Fromm...",4.14,995,"The year 1984 has come and gone, but George Or...","['classics', 'fiction', 'science-fiction', 'fa...",1949.0,Nineteen Eighty-Four,328.0,1956832,"['[George Orwell', 'Marcelo Pen', 'Erich Fromm..."
4,2089,13,"['[George Orwell', 'Marcelo Pen', 'Erich Fromm...",4.14,995,"The year 1984 has come and gone, but George Or...","['classics', 'fiction', 'science-fiction', 'fa...",1949.0,Nineteen Eighty-Four,328.0,1956832,"['[George Orwell', 'Marcelo Pen', 'Erich Fromm..."
...,...,...,...,...,...,...,...,...,...,...,...,...
29995,53110,5220,['Anita Shreve'],3.65,53,In the textile-manufacturing region of New Ham...,"['fiction', 'historical-fiction', 'chick-lit',...",2002.0,Sea Glass,374.0,19585,['Anita Shreve']
29996,53110,5068,['Aldo Leopold'],4.24,74,"First published in 1949, A Sand County Almanac...","['nonfiction', 'science', 'philosophy', 'class...",1949.0,A Sand County Almanac,269.0,16792,['Aldo Leopold']
29997,53110,5395,['Margaret Atwood'],3.66,77,"Now that all the others have run out of air, i...","['fiction', 'historical-fiction', 'fantasy', '...",2005.0,The Penelopiad,198.0,21731,['Margaret Atwood']
29998,53308,8828,['[Andy Cohen]'],3.79,8,"The man behind the ""Real Housewives"" writes ab...","['memoir', 'nonfiction', 'biography']",2012.0,NaN,273.0,9789,['[Andy Cohen]']


In [18]:
candidate_items.to_csv(datapath + "candidate_items_30.csv", index=False)

## fix candidati training

In [65]:
import pandas as pd
import numpy as np
datapath = "tuning/"
test_recommendations = pd.read_csv(datapath + "test_recommendations.csv")
test_recommendations.drop(columns=['rating'], inplace=True)
candidate_items = pd.read_csv(datapath + "candidate_items_70.csv")
app_plots = pd.read_csv(datapath + "apps.csv")
test_recommendations = test_recommendations.merge(app_plots, on="app_id", how="left")
def dcg_score(scores):
    return np.sum(np.divide(np.power(2, scores) - 1, np.log2(np.arange(len(scores), dtype=np.float64) + 2)))

def ndcg_at_10(ranked_relevance_scores, num_relevant_items):
    rank_dcg = dcg_score(ranked_relevance_scores)
    if rank_dcg == 0.0: return 0.0
    ideal_scores = np.ones(min(num_relevant_items, 10))
    ideal_dcg = dcg_score(ideal_scores)
    if ideal_dcg == 0.0: return 0.0
    return rank_dcg / ideal_dcg

test_recommendations.columns, candidate_items.columns

(Index(['user_id', 'app_id', 'authors', 'average_rating', 'books_count', 'plot',
        'genres', 'year', 'new_title', 'pages', 'ratings_count', 'authors_2'],
       dtype='object'),
 Index(['user_id', 'app_id', 'authors', 'average_rating', 'books_count', 'plot',
        'genres', 'year', 'new_title', 'pages', 'ratings_count', 'authors_2'],
       dtype='object'))

In [66]:
all_ndcgs = []
results = []
all_hits = []
for user_id in test_recommendations['user_id'].unique():
    rec = candidate_items[candidate_items['user_id'] == user_id]
    real_rec = test_recommendations[test_recommendations['user_id'] == user_id]
    relevance_scores = rec['app_id'].isin(real_rec['app_id']).astype(int).values
    num_relevant_items = real_rec.shape[0]
    ndcg = ndcg_at_10(relevance_scores, num_relevant_items)
    hits = relevance_scores.sum()
    print(f"User {user_id} - NDCG@10: {ndcg:.4f}")
    all_ndcgs.append(ndcg)
    all_hits.append(hits)
    results.append({
        "user_id": user_id,
        "ndcg_score": ndcg,
        "hits": hits
    })
    #controlla se in results c'è un utente con hits = 0
count=0
for res in results:
    if res['hits'] == 0:
        print(f"User {res['user_id']} has 0 hits")
        count+=1
print(f"Total users with 0 hits: {count}")
print(f"Average NDCG@10: {np.mean(all_ndcgs):.4f}")


User 17978 - NDCG@10: 0.1441
User 22593 - NDCG@10: 0.0000
User 43809 - NDCG@10: 0.0470
User 476 - NDCG@10: 0.0563
User 39161 - NDCG@10: 0.2313
User 35280 - NDCG@10: 0.0000
User 5267 - NDCG@10: 0.4506
User 49755 - NDCG@10: 0.5842
User 43294 - NDCG@10: 0.1290
User 5753 - NDCG@10: 0.3003
User 23863 - NDCG@10: 0.1361
User 33455 - NDCG@10: 0.1483
User 42785 - NDCG@10: 0.2222
User 7405 - NDCG@10: 0.3456
User 1055 - NDCG@10: 0.0365
User 13382 - NDCG@10: 0.5121
User 9659 - NDCG@10: 0.2411
User 8612 - NDCG@10: 0.0821
User 28804 - NDCG@10: 0.2634
User 51600 - NDCG@10: 0.3668
User 19547 - NDCG@10: 0.0926
User 33016 - NDCG@10: 0.1288
User 50977 - NDCG@10: 0.2581
User 41371 - NDCG@10: 0.3367
User 2815 - NDCG@10: 0.1742
User 1301 - NDCG@10: 0.2429
User 40206 - NDCG@10: 0.9483
User 36507 - NDCG@10: 0.2278
User 35537 - NDCG@10: 0.3847
User 36707 - NDCG@10: 0.2946
User 42407 - NDCG@10: 0.2632
User 30843 - NDCG@10: 0.0859
User 31235 - NDCG@10: 0.1361
User 47955 - NDCG@10: 0.1312
User 38362 - NDCG@10: 0.

In [67]:
import pandas as pd
import numpy as np

# Copia del DataFrame originale per non modificarlo direttamente
candidate_items_fixed = candidate_items.copy()

# Lista utenti senza hit
users_with_zero_hits = []

for user_id in test_recommendations['user_id'].unique():
    rec = candidate_items_fixed[candidate_items_fixed['user_id'] == user_id]
    real_rec = test_recommendations[test_recommendations['user_id'] == user_id]
    
    relevance_scores = rec['app_id'].isin(real_rec['app_id']).astype(int).values
    hits = relevance_scores.sum()
    
    if hits == 0:
        users_with_zero_hits.append(user_id)

print(f"Utenti senza hit: {len(users_with_zero_hits)}")

# Correggi i candidate_items per questi utenti
for user_id in users_with_zero_hits:
    # Candidati attuali
    user_candidates = candidate_items_fixed[candidate_items_fixed['user_id'] == user_id]
    
    if user_candidates.empty:
        continue  # salta se non ci sono candidati (caso limite)

    # Un elemento da rimuovere a caso
    random_candidate_index = np.random.choice(user_candidates.index)
    candidate_items_fixed.drop(index=random_candidate_index, inplace=True)
    
    # Prendi un elemento vero dal test set di quell'utente
    user_test_items = test_recommendations[test_recommendations['user_id'] == user_id]
    user_test_sample = user_test_items.sample(n=1, random_state=42)
    
    # Aggiungilo ai candidate_items_fixed
    candidate_items_fixed = pd.concat([candidate_items_fixed, user_test_sample], ignore_index=True)

print(f"✅ Candidati corretti: ora tutti gli utenti hanno almeno un hit garantito.")
users_with_zero_hits=[]
for user_id in test_recommendations['user_id'].unique():
    rec = candidate_items_fixed[candidate_items_fixed['user_id'] == user_id]
    real_rec = test_recommendations[test_recommendations['user_id'] == user_id]
    
    relevance_scores = rec['app_id'].isin(real_rec['app_id']).astype(int).values
    hits = relevance_scores.sum()
    
    if hits == 0:
        users_with_zero_hits.append(user_id)

print(f"Utenti senza hit: {len(users_with_zero_hits)}")
candidate_items_fixed.to_csv(datapath + "candidate_items_70_hits.csv", index=False)

Utenti senza hit: 63
✅ Candidati corretti: ora tutti gli utenti hanno almeno un hit garantito.
Utenti senza hit: 0


In [51]:
candidate_items_fixed.to_csv(datapath + "candidate_items_20_hits.csv", index=False)

In [22]:
import pandas as pd
datapath = "tuning/"
candidate_items = pd.read_csv(datapath + "candidate_items_30_hits.csv")
movies_with_desc = pd.read_csv(datapath + "movies_with_desc.csv")
candidate_items.columns, movies_with_desc.columns

(Index(['user_id', 'app_id', 'title', 'genres', 'new_title', 'year',
        'titolo_match', 'plot', 'imdbId', 'tmdbId'],
       dtype='object'),
 Index(['app_id', 'title', 'tmdbId', 'imdbId', 'desc', 'genres'], dtype='object'))

In [23]:
merged = candidate_items.merge(
    movies_with_desc[['app_id', 'desc']], 
    on='app_id', 
    how='left'
)

print(merged.head())

   user_id  app_id                  title                        genres  \
0        1     364  Lion King, The (1994)  Animation|Children's|Musical   
1       21     364  Lion King, The (1994)  Animation|Children's|Musical   
2       49     364  Lion King, The (1994)  Animation|Children's|Musical   
3       93     364  Lion King, The (1994)  Animation|Children's|Musical   
4      114     364  Lion King, The (1994)  Animation|Children's|Musical   

        new_title  year    titolo_match  \
0  Lion King, The  1994  Lion King, The   
1  Lion King, The  1994  Lion King, The   
2  Lion King, The  1994  Lion King, The   
3  Lion King, The  1994  Lion King, The   
4  Lion King, The  1994  Lion King, The   

                                                plot    imdbId  tmdbId  \
0  Young lion prince Simba, eager to one day beco...  110357.0  8587.0   
1  Young lion prince Simba, eager to one day beco...  110357.0  8587.0   
2  Young lion prince Simba, eager to one day beco...  110357.0  8587

In [24]:
merged.to_csv(datapath + "candidate_items_30_hits.csv", index=False)

In [25]:
candidate_items_30_hits = pd.read_csv(datapath + "candidate_items_30_hits.csv")
candidate_items_30_hits.head()

,user_id,app_id,title,genres,new_title,year,titolo_match,plot,imdbId,tmdbId,desc
0,1,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0,"Young lion prince Simba, eager to one day beco..."
1,21,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0,"Young lion prince Simba, eager to one day beco..."
2,49,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0,"Young lion prince Simba, eager to one day beco..."
3,93,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0,"Young lion prince Simba, eager to one day beco..."
4,114,364,"Lion King, The (1994)",Animation|Children's|Musical,"Lion King, The",1994,"Lion King, The","Young lion prince Simba, eager to one day beco...",110357.0,8587.0,"Young lion prince Simba, eager to one day beco..."


In [14]:
import pandas as pd
datapath = "tuning/"
candidate_items_path = datapath + "candidate/candidate_items_30_hits.csv"
candidate_items = pd.read_csv(candidate_items_path)
app_plots = pd.read_csv(datapath + "apps_fixed2.csv")
candidate_items.drop(columns=['new_title'], inplace=True)
candidate_items = candidate_items.merge(app_plots[['app_id', 'new_title']], on='app_id', how='left')
candidate_items.to_csv(candidate_items_path, index=False)

In [ ]:
# add the column new_title from app_plots to candidate_items
candidate_items = candidate_items.merge(app_plots[['app_id', 'new_title']], on='app_id', how='left')
candidate_items.to_csv(datapath + "candidate/candidate_items_10_hits.csv", index=False)

,user_id,app_id,authors,average_rating,books_count,plot,genres,year,pages,ratings_count,authors_2,new_title
0,167,178,['Sylvia Plath'],3.98,246,The Bell Jar chronicles the crack-up of Esther...,"['classics', 'fiction', 'psychology', 'poetry']",1963.0,294.0,401605,['Sylvia Plath'],The Bell Jar
1,8707,178,['Sylvia Plath'],3.98,246,The Bell Jar chronicles the crack-up of Esther...,"['classics', 'fiction', 'psychology', 'poetry']",1963.0,294.0,401605,['Sylvia Plath'],The Bell Jar
2,9661,178,['Sylvia Plath'],3.98,246,The Bell Jar chronicles the crack-up of Esther...,"['classics', 'fiction', 'psychology', 'poetry']",1963.0,294.0,401605,['Sylvia Plath'],The Bell Jar
3,12285,178,['Sylvia Plath'],3.98,246,The Bell Jar chronicles the crack-up of Esther...,"['classics', 'fiction', 'psychology', 'poetry']",1963.0,294.0,401605,['Sylvia Plath'],The Bell Jar
4,14950,178,['Sylvia Plath'],3.98,246,The Bell Jar chronicles the crack-up of Esther...,"['classics', 'fiction', 'psychology', 'poetry']",1963.0,294.0,401605,['Sylvia Plath'],The Bell Jar
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,51329,361,"['Charles Dickens', 'George Cruikshank', 'Phil...",3.85,1787,A gripping portrayal of London's dark criminal...,"['classics', 'fiction', 'historical-fiction']",1838.0,608.0,230181,"['Charles Dickens', 'George Cruikshank', 'Phil...",Oliver Twist
9996,39400,280,['Lauren Oliver'],3.99,108,There is an alternate cover edition for this I...,"['young-adult', 'romance', 'science-fiction', ...",2011.0,441.0,256975,['Lauren Oliver'],"Delirium (Delirium, #1)"
9997,28258,9718,['[Chris Cleave]'],3.48,25,When a massive suicide bomb explodes at a Lond...,"['fiction', 'contemporary', 'thriller']",2012.0,239.0,6058,['[Chris Cleave]'],"Happier at Home: Kiss More, Jump More, Abandon..."
9998,27762,2243,['[A.J. Banner]'],3.33,14,From a phenomenal new voice in suspense fictio...,"['fiction', 'mystery', 'thriller', 'suspense',...",2015.0,NaN,37604,['[A.J. Banner]'],The Good Neighbor


In [21]:
# verifica quanti titoli in candidate_items sono NaN
no_title = candidate_items.loc[ candidate_items['new_title'].isna(), 'app_id' ]
no_title = set( no_title )
len( no_title )

0